<a href="https://colab.research.google.com/github/Dandhalf/DACON_contest2022/blob/main/%EB%8D%B0%EC%9D%B4%EC%BD%98_%EC%86%8C%EB%B9%84_%EC%98%88%EC%B8%A1_%EA%B2%BD%EC%A7%84%EB%8C%80%ED%9A%8C_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 해당 데이터에서 질문
- 특성 값 중에 시간에 대한 컬럼이 있을 경우에는 어떻게 다룰 것인가?
- 이번 대회에서는 예측값을 제출 해야함

# 00. 분석 준비하기
## 00- 1 chage directory & zip 파일 압축해제

In [ ]:
# from google.colab import drive
# # drive.mount('/content/drive')

In [ ]:
# %cd "/content/drive/MyDrive/Colab Notebooks"
# !unzip "소비 예측 경진대회-20220504T134719Z-001.zip"

/content/drive/MyDrive/Colab Notebooks
Archive:  소비 예측 경진대회-20220504T134719Z-001.zip
  inflating: 소비 예측 경진대회/test.csv  
  inflating: 소비 예측 경진대회/train.csv  
  inflating: 소비 예측 경진대회/sample_submission.csv  


## 00-2. 라이브러리 import

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## 00-3. 데이터불러오기

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/소비 예측 경진대회/train.csv")

#01. 데이터 탐색하기

In [ ]:
df.head()

,id,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,NumDealsPurchases,...,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response,target
0,0,1974,Master,Together,46014.0,1,1,21-01-2013,21,10,...,8,7,0,0,0,0,0,0,0,541
1,1,1962,Graduation,Single,76624.0,0,1,24-05-2014,68,1,...,7,1,1,0,0,0,0,0,0,899
2,2,1951,Graduation,Married,75903.0,0,1,08-04-2013,50,2,...,9,3,0,0,0,0,0,0,0,901
3,3,1974,Basic,Married,18393.0,1,0,29-03-2014,2,2,...,3,8,0,0,0,0,0,0,0,50
4,4,1946,PhD,Together,64014.0,2,1,10-06-2014,56,7,...,5,7,0,0,0,1,0,0,0,444


- ... 으로 볼 수 있는 columns의 수가 제한되어 있음으로 전체 볼 수 있도록 아래 코드를 돌려주자

In [ ]:
pd.set_option('display.max_columns', 50) # 'columns 50개 까지 보여줘' 라는 뜻

In [ ]:
df.tail(3)

,id,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response,target
1105,1105,1975,Master,Married,37368.0,1,0,16-12-2013,4,1,1,0,2,6,1,0,0,0,0,0,0,30
1106,1106,1974,Graduation,Divorced,53034.0,1,1,30-05-2013,30,8,6,1,7,8,0,0,0,0,0,0,0,447
1107,1107,1952,PhD,Divorced,46610.0,0,2,29-10-2012,8,6,4,1,6,6,0,0,0,0,0,0,1,302


In [ ]:
df.shape

(1108, 22)

### 1108데이터는 17480 개체와 특성은 22개 가지고 있음.

In [ ]:
df['Education'].unique()

array(['Master', 'Graduation', 'Basic', 'PhD', '2n Cycle'], dtype=object)

In [ ]:
df.keys()

Index(['id', 'Year_Birth', 'Education', 'Marital_Status', 'Income', 'Kidhome',
       'Teenhome', 'Dt_Customer', 'Recency', 'NumDealsPurchases',
       'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases',
       'NumWebVisitsMonth', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5',
       'AcceptedCmp1', 'AcceptedCmp2', 'Complain', 'Response', 'target'],
      dtype='object')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1108 entries, 0 to 1107
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   1108 non-null   int64  
 1   Year_Birth           1108 non-null   int64  
 2   Education            1108 non-null   object 
 3   Marital_Status       1108 non-null   object 
 4   Income               1108 non-null   float64
 5   Kidhome              1108 non-null   int64  
 6   Teenhome             1108 non-null   int64  
 7   Dt_Customer          1108 non-null   object 
 8   Recency              1108 non-null   int64  
 9   NumDealsPurchases    1108 non-null   int64  
 10  NumWebPurchases      1108 non-null   int64  
 11  NumCatalogPurchases  1108 non-null   int64  
 12  NumStorePurchases    1108 non-null   int64  
 13  NumWebVisitsMonth    1108 non-null   int64  
 14  AcceptedCmp3         1108 non-null   int64  
 15  AcceptedCmp4         1108 non-null   i

### 누락값을 확인했으나 없음
### Education, Marital_Status, Dt_Customer : data type - object

In [ ]:
df.isnull().sum()

id                     0
Year_Birth             0
Education              0
Marital_Status         0
Income                 0
Kidhome                0
Teenhome               0
Dt_Customer            0
Recency                0
NumDealsPurchases      0
NumWebPurchases        0
NumCatalogPurchases    0
NumStorePurchases      0
NumWebVisitsMonth      0
AcceptedCmp3           0
AcceptedCmp4           0
AcceptedCmp5           0
AcceptedCmp1           0
AcceptedCmp2           0
Complain               0
Response               0
target                 0
dtype: int64

## 추가부분 : 시간자료 'Dt_Customer' 컬럼 처리
- 'Dt_Customer' 컬럼에서 year 정보만 따로 떼기

In [ ]:
df['Dt_Customer']

0       21-01-2013
1       24-05-2014
2       08-04-2013
3       29-03-2014
4       10-06-2014
           ...    
1103    31-03-2013
1104    21-10-2013
1105    16-12-2013
1106    30-05-2013
1107    29-10-2012
Name: Dt_Customer, Length: 1108, dtype: object

In [ ]:
df['Dt_Customer_year'] = pd.DatetimeIndex(df['Dt_Customer']).year
df['Dt_Customer_year']

0       2013
1       2014
2       2013
3       2014
4       2014
        ... 
1103    2013
1104    2013
1105    2013
1106    2013
1107    2012
Name: Dt_Customer_year, Length: 1108, dtype: int64

# 02 데이터 전처리
## 02-1. X, y 나누기

- target 제품 총 소비량 예측하는 것이 목표 

In [ ]:
X = df.drop(["id","Dt_Customer","target"], axis = 1)
y = df["target"]

In [ ]:
# # 연도와 관련 데이터 빼봄
# X = df.drop(["id","Dt_Customer","Dt_Customer_year","target"], axis = 1)
# y = df["target"]

## 02-2. data(X) 범주형 변수 가변수로 처리
- X의 object data type 값들 pd.get_dummies로 더미변수로 변환

### 질문
- 시간 자료는 어떻게 처리하는 것이 좋은가? - 'Dt_Customer'

In [ ]:
X.keys()

Index(['Year_Birth', 'Education', 'Marital_Status', 'Income', 'Kidhome',
       'Teenhome', 'Recency', 'NumDealsPurchases', 'NumWebPurchases',
       'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth',
       'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1',
       'AcceptedCmp2', 'Complain', 'Response', 'Dt_Customer_year'],
      dtype='object')

In [ ]:
X = pd.get_dummies(X[['Year_Birth', 'Education', 'Marital_Status', 'Income', 'Kidhome',
       'Teenhome', 'Recency', 'NumDealsPurchases',
       'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases',
       'NumWebVisitsMonth', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5',
       'AcceptedCmp1', 'AcceptedCmp2', 'Complain', 'Response', 'Dt_Customer_year']],
       columns = ['Education', 'Marital_Status'], drop_first = True)

In [ ]:
# # 연도와 관련 데이터 빼봄
# X = pd.get_dummies(X[['Year_Birth', 'Education', 'Marital_Status', 'Income', 'Kidhome',
#        'Teenhome', 'Recency', 'NumDealsPurchases',
#        'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases',
#        'NumWebVisitsMonth', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5',
#        'AcceptedCmp1', 'AcceptedCmp2', 'Complain', 'Response']],
#        columns = ['Education', 'Marital_Status'], drop_first = True)

## 02-3. 훈련데이터와 테스트 데이터 나눔

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 2022)

## 02-4. data(X) 수치형 자료 표준화
- train 데이터 세트의 표준화 포멧을 test 데이터 세트에 적용해야 함으로 순서가 이러함
- 'Dt_Customer' : 표준화에서 제외
- 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1', 'AcceptedCmp2', 'Complain', 'Response' : 데이터 타입이 0 or 1 이라서 표준화 안함

In [ ]:
from sklearn.preprocessing import StandardScaler
stdsc = StandardScaler()

In [ ]:
# 아래 코드는 .iloc 사용하지 않고도 표준화 가능함을 보여주는 예시

# X_train[['Income', 'Recency']] = stdsc.fit_transform(X_train[['Income', 'Recency']])
# X_test.iloc[:,[0,1,2,3,4,5]] = stdsc.transform(X_test.iloc[:,[0,1,2,3,4,5]])

In [ ]:
X_train[['Year_Birth', 'Income', 'Kidhome',
       'Teenhome', 'Recency', 'NumDealsPurchases',
       'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases',
       'NumWebVisitsMonth', 'Dt_Customer_year']] = stdsc.fit_transform(X_train[['Year_Birth', 'Income', 'Kidhome',
       'Teenhome', 'Recency', 'NumDealsPurchases',
       'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases',
       'NumWebVisitsMonth', 'Dt_Customer_year']])

In [ ]:
X_train.head()

,Year_Birth,Income,Kidhome,Teenhome,Dt_Customer,Recency,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response,Dt_Customer_year,Education_Basic,Education_Graduation,Education_Master,Education_PhD,Marital_Status_Alone,Marital_Status_Divorced,Marital_Status_Married,Marital_Status_Single,Marital_Status_Together,Marital_Status_Widow,Marital_Status_YOLO
453,0.690771,-0.742363,1.040307,-0.925898,16-04-2014,1.046336,-0.185655,-0.420402,-0.960742,-0.884755,0.268087,0,0,0,0,0,0,0,1.487143,0,0,1,0,0,0,0,1,0,0,0
712,-1.195405,1.561490,-0.781258,-0.925898,21-03-2014,-1.516403,-0.693340,-0.773126,0.487522,1.259539,-1.800949,0,1,1,1,1,0,1,1.487143,0,1,0,0,0,1,0,0,0,0,0
674,1.592855,-1.131002,1.040307,-0.925898,28-04-2014,-0.650613,-0.693340,-1.125850,-0.960742,-0.578427,-0.973335,0,0,0,0,0,0,0,1.487143,0,1,0,0,0,0,0,1,0,0,0
806,-0.293321,0.905444,-0.781258,-0.925898,21-08-2013,-0.823771,-0.693340,1.343217,-0.236610,1.259539,-0.559527,0,0,0,0,0,0,0,-0.020418,0,1,0,0,0,0,0,0,1,0,0
346,0.116717,0.708257,-0.781258,0.881031,10-11-2012,-0.615981,-0.185655,0.990493,1.935786,-0.272099,-0.145720,0,1,0,0,0,0,0,-1.527980,0,1,0,0,0,0,0,1,0,0,0


In [ ]:
X_test[['Year_Birth', 'Income', 'Kidhome',
       'Teenhome', 'Recency', 'NumDealsPurchases',
       'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases',
       'NumWebVisitsMonth', 'Dt_Customer_year']] = stdsc.transform(X_test[['Year_Birth', 'Income', 'Kidhome',
       'Teenhome', 'Recency', 'NumDealsPurchases',
       'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases',
       'NumWebVisitsMonth', 'Dt_Customer_year']])

# 03. 모델 구축하기
## 03-1. 모델선정
- LinearRegression

In [ ]:
from sklearn.linear_model import LinearRegression
slr = LinearRegression()
slr.fit(X_train, y_train)

y_train_pred = slr.predict(X_train)
y_test_pred = slr.predict(X_test)

### 03-2. 모델의 성능 평가
- MSE : 평균 제곱 오차
- 결정계수

In [ ]:
from sklearn.metrics import mean_squared_error
print("훈련 세트 데이터 MSE : {:.3f}".format(mean_squared_error(y_train, y_train_pred)))
print("테스트 세트 데이터 MSE : {:.3f}".format(mean_squared_error(y_test, y_test_pred)))

훈련 세트 데이터 MSE : 63478.310
테스트 세트 데이터 MSE : 93636369375253.016


In [ ]:
from sklearn.metrics import r2_score
print("훈련 R^2 : %.3f, 테스트 R^2 :  %.3f" %
      (r2_score(y_train, y_train_pred),
      r2_score(y_test, y_test_pred)))

훈련 R^2 : 0.823, 테스트 R^2 :  0.837


## RANSAC 모델 사용해보자

In [ ]:
from sklearn.linear_model import RANSACRegressor
ransac = RANSACRegressor(LinearRegression(),
                          max_trials = 100,
                          min_samples = 50,
                          loss = 'absolute_loss',
                          residual_threshold = 5.0,
                          random_state=0)
ransac.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_ransac.py:358: FutureWarning: The loss 'absolute_loss' was deprecated in v1.0 and will be removed in version 1.2. Use `loss='absolute_error'` which is equivalent.
  FutureWarning,


RANSACRegressor(base_estimator=LinearRegression(), loss='absolute_loss',
                min_samples=50, random_state=0, residual_threshold=5.0)

In [ ]:
y_train_pred = ransac.predict(X_train)
y_test_pred = ransac.predict(X_test)
print("훈련 R^2 : %.3f, 테스트 R^2 :  %.3f" %
      (r2_score(y_train, y_train_pred),
      r2_score(y_test, y_test_pred)))

훈련 R^2 : 0.666, 테스트 R^2 :  0.680


# 04. 최적 모델 찾기


# 05. final test 데이터로 예측하기
## 05-1. 데이터 불러오기

In [ ]:
final_test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/소비 예측 경진대회/test.csv")

In [ ]:
final_test.head()

,id,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response
0,0,1979,Graduation,Single,57537.0,1,0,10-06-2013,83,4,4,3,8,4,0,0,0,0,0,0,0
1,1,1955,Graduation,Together,65748.0,0,1,15-04-2013,58,2,2,4,10,1,0,0,0,0,0,1,0
2,2,1984,Graduation,Married,55434.0,1,0,05-10-2013,21,3,5,3,13,4,0,0,0,0,0,0,0
3,3,1960,Graduation,Married,67716.0,0,1,10-01-2013,8,4,7,4,13,5,0,1,0,0,0,0,0
4,4,1976,Graduation,Together,85606.0,0,1,12-11-2012,89,2,6,7,9,3,0,0,0,1,0,0,0


## id 컬럼은 제출 파일 만들때 따로 써야 함으로 떼어놓음

In [ ]:
final_id = final_test['id']
final_id.head()

0    0
1    1
2    2
3    3
4    4
Name: id, dtype: int64

## 05-2. final test 데이터 전처리하기
1) 년도 추출  

In [ ]:
final_test['Dt_Customer_year'] = pd.DatetimeIndex(final_test['Dt_Customer']).year
final_test['Dt_Customer_year']

NameError: ignored

2) 일부 컬럼 drop  
3) 범주형 변수 -> 더미 변수화  
4) 수치형 변수 -> 표준화

In [ ]:
final_test = df.drop(["id","Dt_Customer","target"], axis = 1)

final_test = pd.get_dummies(final_test[['Year_Birth', 'Education', 'Marital_Status', 'Income', 'Kidhome',
       'Teenhome', 'Recency', 'NumDealsPurchases',
       'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases',
       'NumWebVisitsMonth', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5',
       'AcceptedCmp1', 'AcceptedCmp2', 'Complain', 'Response', 'Dt_Customer_year']],
       columns = ['Education', 'Marital_Status'], drop_first = True)

final_test[['Year_Birth', 'Income', 'Kidhome',
       'Teenhome', 'Recency', 'NumDealsPurchases',
       'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases',
       'NumWebVisitsMonth', 'Dt_Customer_year']] = stdsc.transform(final_test[['Year_Birth', 'Income', 'Kidhome',
       'Teenhome', 'Recency', 'NumDealsPurchases',
       'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases',
       'NumWebVisitsMonth', 'Dt_Customer_year']])

## 05-3. 모델에 넣어 예측 값 얻기

In [ ]:
submission_predictions = slr.predict(final_test)

prediction_file = pd.DataFrame({'id' : final_id,
                               'target' : submission_predictions})
prediction_file.head(6)

,id,target
0,0,503.553526
1,1,1276.314362
2,2,1124.373483
3,3,-35.612311
4,4,580.239330
5,5,531.972490


In [ ]:
prediction_file.to_csv('소비 예측 경진대회.csv', index = False)